In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 https://developer.download.nvidia.com/compute/c

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Cardio_Data_CleanUp").getOrCreate()

### Load Data into Spark DataFrame

In [4]:
import os
os.getcwd()

'/content'

In [5]:
from pyspark import SparkFiles
csv_file_path = "/content/Resources/Cardio_Data.csv"
cardio_df_original = spark.read.csv(csv_file_path, header=True)
cardio_df_original.show()
cardio_df_original.printSchema()

+-----+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|   id|  age|gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|
+-----+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|82567|18804|     2|   165|    10|  180| 1100|          2|   2|    0|   0|     1|     1|
|48318|21582|     2|   178|    11|  130|   90|          1|   1|    0|   0|     1|     1|
|85931|21855|     1|   162|    21|  120|   80|          2|   1|    0|   0|     1|     1|
|42156|20408|     2|   177|    22|  120|   80|          1|   1|    1|   1|     1|     0|
|38312|23284|     1|   157|    23|  110|   80|          1|   1|    0|   0|     1|     0|
|48976|14664|     2|   128|    28|  120|   80|          1|   1|    0|   0|     1|     0|
|86650|18875|     1|   171|    29|  110|   70|          2|   1|    0|   0|     1|     1|
| 5306|15400|     1|   120|    30|  110|   70|          1|   1|    0|   0|     1|     0|
|26503|18140|     1| 

### Exploratory Data Analysis 

In [6]:
# Change the column names to be more meaningful
cardio_df = cardio_df_original.withColumnRenamed("ap_hi", "systolic_bp").withColumnRenamed("ap_lo", "diastolic_bp")\
                              .withColumnRenamed("gluc", "glucose").withColumnRenamed("smoke", "smoker")\
                              .withColumnRenamed("alco", "alcohol_intake")
cardio_df.toPandas().head(5)

,id,age,gender,height,weight,systolic_bp,diastolic_bp,cholesterol,glucose,smoker,alcohol_intake,active,cardio
0,82567,18804,2,165,10,180,1100,2,2,0,0,1,1
1,48318,21582,2,178,11,130,90,1,1,0,0,1,1
2,85931,21855,1,162,21,120,80,2,1,0,0,1,1
3,42156,20408,2,177,22,120,80,1,1,1,1,1,0
4,38312,23284,1,157,23,110,80,1,1,0,0,1,0


In [7]:
# Summary statistics of the continuous variables
cardio_df.select("id", "age", "height", "weight", "systolic_bp", "diastolic_bp").describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+------------------+
|summary|                id|               age|            height|            weight|       systolic_bp|      diastolic_bp|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+
|  count|             70000|             70000|             70000|             70000|             70000|             70000|
|   mean|        49972.4199|19468.865814285713|164.35922857142856| 74.20568999999998| 128.8172857142857| 96.63041428571428|
| stddev|28851.302323172928|2467.2516672413917| 8.210126364538551|14.395756678511473|154.01141945609032|188.47253029639106|
|    min|                 0|             10798|               100|                10|              -100|               -70|
|    max|             99999|             23713|                99|              99.9|                99|                99|
+-------

In [8]:
# Find the correlation between the continuous variables
from pyspark.mllib.stat import Statistics

# select variables to check correlation
cardio_df_features = cardio_df.select("id", "age", "height","weight","systolic_bp","diastolic_bp") 

# create RDD table for correlation calculation
rdd_table = cardio_df_features.rdd.map(lambda row: row[0:])

# get the correlation matrix
corr_mat=Statistics.corr(rdd_table, method="pearson")
print(corr_mat)

[[ 1.          0.00345653 -0.00303782 -0.00182999  0.0033556  -0.00252928]
 [ 0.00345653  1.         -0.08151535  0.05368404  0.02076432  0.01764743]
 [-0.00303782 -0.08151535  1.          0.29096783  0.00548776  0.00615047]
 [-0.00182999  0.05368404  0.29096783  1.          0.03070221  0.04370977]
 [ 0.0033556   0.02076432  0.00548776  0.03070221  1.          0.01608551]
 [-0.00252928  0.01764743  0.00615047  0.04370977  0.01608551  1.        ]]


##### At this time we notice no substantial correlation between any of the continuous variables.
#####  Except for a slight correlation between height and weight.
##### (We have observed improbable values for height and weight, and this could have impacted the correlation)

In [9]:
# Check for null values
Dict_Null = {col:cardio_df.filter(cardio_df[col].isNull()).count() for col in cardio_df.columns}
Dict_Null

{'active': 0,
 'age': 0,
 'alcohol_intake': 0,
 'cardio': 0,
 'cholesterol': 0,
 'diastolic_bp': 0,
 'gender': 0,
 'glucose': 0,
 'height': 0,
 'id': 0,
 'smoker': 0,
 'systolic_bp': 0,
 'weight': 0}

In [10]:
# Check for duplicate rows. We will do this by counting the number of distinct rows.
cardio_df.distinct().count()

70000

##### The number of distinct rows id 70,000. So we know that this DataFrame has no duplicate rows

In [11]:
# Check the distinct values of the categorical variables
# https://stackoverflow.com/questions/64805788/get-distinct-values-of-multiple-columns
from pyspark.sql.functions import collect_set
from pyspark.sql.functions import col

In [12]:
columns = ["gender", "cholesterol", "glucose", "smoker", "alcohol_intake", "active", "cardio"]
columnExprs = map(lambda c: collect_set(col(c)).alias(c), columns)
cardio_df.select(*columnExprs).show()

+------+-----------+---------+------+--------------+------+------+
|gender|cholesterol|  glucose|smoker|alcohol_intake|active|cardio|
+------+-----------+---------+------+--------------+------+------+
|[1, 2]|  [3, 1, 2]|[3, 1, 2]|[1, 0]|        [1, 0]|[1, 0]|[1, 0]|
+------+-----------+---------+------+--------------+------+------+



In [13]:
# Check for frequencies of categorical variables.
cardio_df.groupBy("gender").count().show()
cardio_df.groupBy("cholesterol").count().show()
cardio_df.groupBy("glucose").count().show()
cardio_df.groupBy("smoker").count().show()
cardio_df.groupBy("alcohol_intake").count().show()
cardio_df.groupBy("active").count().show()
cardio_df.groupBy("cardio").count().show()

+------+-----+
|gender|count|
+------+-----+
|     1|45530|
|     2|24470|
+------+-----+

+-----------+-----+
|cholesterol|count|
+-----------+-----+
|          3| 8066|
|          1|52385|
|          2| 9549|
+-----------+-----+

+-------+-----+
|glucose|count|
+-------+-----+
|      3| 5331|
|      1|59479|
|      2| 5190|
+-------+-----+

+------+-----+
|smoker|count|
+------+-----+
|     0|63831|
|     1| 6169|
+------+-----+

+--------------+-----+
|alcohol_intake|count|
+--------------+-----+
|             0|66236|
|             1| 3764|
+--------------+-----+

+------+-----+
|active|count|
+------+-----+
|     0|13739|
|     1|56261|
+------+-----+

+------+-----+
|cardio|count|
+------+-----+
|     0|35021|
|     1|34979|
+------+-----+



### Data Processing on continous variables

In [14]:
# Filter based on height. Keep height between 135 - 215cm
cardio_df = cardio_df.filter((cardio_df["height"] >=135) & (cardio_df["height"] <=215))

In [15]:
# The shape of the dataframe after filtering out the improbable values of height
print((cardio_df.count(), len(cardio_df.columns)))

(69883, 13)


In [16]:
# Filter based on weight. Keep height between 25 - 200kg
cardio_df = cardio_df.filter((cardio_df["weight"] >=25) & (cardio_df["weight"] <=200))

In [17]:
# The shape of the dataframe after filtering out the improbable values of weight
print((cardio_df.count(), len(cardio_df.columns)))

(69878, 13)


In [18]:
# Change the signs of negative values of systolic and diastolic bp
from pyspark.sql.functions import abs
cardio_df = cardio_df.withColumn("systolic_bp", abs(cardio_df["systolic_bp"])).withColumn("diastolic_bp", abs(cardio_df["diastolic_bp"]))

In [19]:
# Check for presence of negative values in the columns
cardio_df.filter((cardio_df["diastolic_bp"] <0)).groupby("diastolic_bp").count().show()
cardio_df.filter((cardio_df["systolic_bp"] <0)).groupby("systolic_bp").count().show()


+------------+-----+
|diastolic_bp|count|
+------------+-----+
+------------+-----+

+-----------+-----+
|systolic_bp|count|
+-----------+-----+
+-----------+-----+



In [20]:
# Filter based on systolic_bp values between 80 and 180. 
cardio_df = cardio_df.filter((cardio_df["systolic_bp"] >=80) & (cardio_df["systolic_bp"] <=180))

In [21]:
# Filter based on diastolic_bp values between 40 and 120. 
cardio_df = cardio_df.filter((cardio_df["diastolic_bp"] >=40) & (cardio_df["diastolic_bp"] <=120))

In [22]:
# The shape of the dataframe after filtering out the improbable values of systolic and diastolic bp
print((cardio_df.count(), len(cardio_df.columns)))

(68297, 13)


In [23]:
# Turn age values from days to years
import pyspark.sql.functions as F

In [24]:
cardio_df = cardio_df.withColumn("Age", (F.col("age") / 365).cast('int')) 

In [25]:
cardio_df.show(5)

+-----+---+------+------+------+-----------+------------+-----------+-------+------+--------------+------+------+
|   id|Age|gender|height|weight|systolic_bp|diastolic_bp|cholesterol|glucose|smoker|alcohol_intake|active|cardio|
+-----+---+------+------+------+-----------+------------+-----------+-------+------+--------------+------+------+
|86650| 51|     1|   171|    29|      110.0|        70.0|          2|      1|     0|             0|     1|     1|
|26503| 49|     1|   160|    30|      120.0|        80.0|          1|      1|     0|             0|     1|     1|
|59853| 58|     1|   143|    30|      103.0|        61.0|          2|      1|     0|             0|     1|     0|
|24167| 47|     2|   170|    31|      150.0|        90.0|          2|      2|     0|             0|     1|     1|
|31439| 42|     1|   146|    32|      100.0|        70.0|          1|      1|     0|             0|     0|     0|
+-----+---+------+------+------+-----------+------------+-----------+-------+------+----

##### The variable id will not be useful for further analysis, as it yeilds no meaningful data contributing to heart disease.
##### The column id will therefore be dropped.

In [27]:
cardio_df = cardio_df.drop("id")

In [28]:
cardio_df.toPandas().head(5)

,Age,gender,height,weight,systolic_bp,diastolic_bp,cholesterol,glucose,smoker,alcohol_intake,active,cardio
0,51,1,171,29,110.0,70.0,2,1,0,0,1,1
1,49,1,160,30,120.0,80.0,1,1,0,0,1,1
2,58,1,143,30,103.0,61.0,2,1,0,0,1,0
3,47,2,170,31,150.0,90.0,2,2,0,0,1,1
4,42,1,146,32,100.0,70.0,1,1,0,0,0,0


### Storing data into output csv file

In [31]:
# Write to output csvfile.
# First convert to a pandas_df
cardio_cleaned_df = cardio_df.toPandas()

# Store result
cardio_cleaned_df.to_csv("/content/Resources/cardio_data_cleaned.csv", index=False)